# Trabalho Prático 3  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Exercício 1

No capítulo 5 dos apontamentos é descrito o chamado Hidden Number Problem. No capítulo 8 dos apontamentos é discutida um artigo de  Nguyen & Shparlinsk , onde se propõem reduções do HNP a problemas difíceis em reticulados. Neste trabalho pretende-se construir, com a ajuda do Sagemath, uma implementação da solução discutida nos apontamentos para resolver o HNP com soluções aproximadas dos problemas em reticulados.

In [232]:
%pip install sagemath-standard
from sage.all import *

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Etapa 1: Construção da matriz 𝐺.

In [233]:
# Função para construir a matriz G
def build_G_matrix(p, x_values, A, u_values, M):
    n = len(x_values)
    m = n + 2
    G = Matrix(QQ, m, m)

    # Preenchendo os elementos de G
    for i in range(m):
        for j in range(m):
            if i == m - 2 and j < n:
                G[i, j] = x_values[j]
            elif i == m - 1 and j < n:
                G[i, j] = -B * u_values[j]
            elif i == m - 1 and j == m - 1:
                G[i, j] = M
            elif i == m - 1 and j == n:
                G[i, j] = 0
            elif i == m - 1 and j == n + 1:
                G[i, j] = M
            elif i < n and j < n:
                G[i, j] = 0 if i != j else p
            elif i == n and j == n:
                G[i, j] = A
            elif i == n + 1 and j == n + 1:
                G[i, j] = 1
            else:
                G[i, j] = 0

    return G

Etapa 2: Construção do vetor alvo 𝑦.

In [234]:
# Função para construir o vetor alvo y
def build_y_vector(p, u_values, B):
    n = len(u_values)
    y = [B * u % p for u in u_values] + [0, 0]  # Construindo o vetor y

    return vector(y)

Etapa 3: Resolução do Problema do Menor Vetor em uma Reticulado (BDD).

In [235]:
# Função para resolver o BDD
def solve_BDD(G, y, B):
    # Ensure G and y have compatible dimensions
    if len(G[0]) == len(y):
        # Convertendo y para um vetor de SageMath
        y_vector = vector(y)

        # Construindo o reticulado e reduzindo a base
        L = Matrix(G).LLL()

        # Calculando o ponto mais próximo de y no reticulado
        closest_point = L * y_vector

        # Arredondando o ponto mais próximo
        rounded_point = vector(ZZ, [round(coord) for coord in closest_point])
        #print(rounded_point)
        #print(y_vector)
        #print(vector(rounded_point - y_vector).norm())
        #print(B)
        # Verificando se a distância é menor que B
        if vector(rounded_point - y_vector).norm() <= B:
            return rounded_point
        else:
            return None
    else:
        print("G and y have incompatible dimensions")
        # Handle the case when G and y have incompatible dimensions

Etapa 4: Recuperação do segredo 𝑠 a partir do vetor solução.

In [236]:
# Função para recuperar o segredo
def recover_secret(solution, A):
    # Extraindo a última componente do vetor solução
    s_A = solution[-1]

    # Arredondando para obter o valor de s
    s = ceil(s_A / A)

    return s

Etapa 5: Avaliação da precisão da solução.

In [237]:
# Função para avaliar a precisão da solução
def evaluate_solution(y, s, p, B):
    # Calculando o vetor obtido a partir do segredo recuperado
    y_obtained = vector([s * xi % p for xi in x_values] + [0, 0])

    # Calculando a distância entre y e y_obtained
    distance = vector(y - y_obtained).norm()

    # Verificando se a distância é menor ou igual a B
    if distance <= B:
        return True, distance
    else:
        return False, distance

Definir os parâmetros a usar para resolver

In [238]:
# Parâmetros do problema
p = next_prime(41)  # Primo p
x_values = [3, 5, 0]  # Valores xi
u_values = [2, 0, 5]  # Valores ui
A = 1/2  # Valor A
B = 10^5  # Parâmetro B
M = 10^7  # Valor M

In [239]:
# Construindo a matriz G
G = build_G_matrix(p, x_values, A, u_values, M)

In [240]:
# Construindo o vetor alvo y
y = build_y_vector(p, u_values, B)

In [241]:
# Resolvendo o BDD
solution = solve_BDD(G, y, B)
print("Solution:", solution)

Solution: (21, 35, 56, 1677, 128)


In [242]:
# Verificando se a solução foi encontrada
if solution:
    # Recuperando o segredo
    s = recover_secret(solution, A)

    # Avaliando a precisão da solução
    is_within_limit, distance = evaluate_solution(y, s, p, B)

    # Exibindo resultados
    if is_within_limit:
        print("Solução encontrada dentro do limite B.")
        print("Segredo recuperado:", s)
        print("Distância entre y e y_obtained:", distance)
    else:
        print("Solução encontrada, mas está fora do limite B.")
        print("Segredo recuperado:", s)
        print("Distância entre y e y_obtained:", distance)
else:
    print("Não foi possível encontrar uma solução dentro do limite B.")

Solução encontrada dentro do limite B.
Segredo recuperado: 256
Distância entre y e y_obtained: 3*sqrt(390)
